In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import gym

In [3]:
from gym.envs.registration import register

register(
    id="FrozenLakeNoSlippery-v0",
    entry_point="gym.envs.toy_text:FrozenLakeEnv",
    kwargs={"map_name": "4x4", "is_slippery":False},
    max_episode_steps=100,
    reward_threshold=0.78
)

In [5]:
env = gym.make("FrozenLakeNoSlippery-v0")
env.reset()

for step in range(5):
    env.render()
    action = env.action_space.sample()
    env.step(action)

env.close()


SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
